In [1]:
import moldesign as mdt
from moldesign import units as u

OpenMM could not be imported; using remote docker container
PDBFixer could not be imported; using remote docker container
PySCF not installed; using remote docker container
Reading configuration from /Users/t_leeday/.moldesign/moldesign.yml


In [2]:
from moldesign.models.lammps_model import LAMMPSPotential
import parmed as med

In [3]:
mol = mdt.from_pdb('1yu8')


600
Finished initializing


In [ ]:
mol = mdt.Molecule([res for res in mol.residues if res.type == "protein"])

64
About to copy!
Done
Finished initializing


In [ ]:
mdmol = mdt.assign_forcefield(mol)

print "Done parameterizing"



508
About to copy!
Done
Finished initializing
Connecting to docker host at unix://var/run/docker.sock... done


In [ ]:
mdmol.set_energy_model(LAMMPSPotential)


In [ ]:
hydrogens = [atom for atom in mdmol.atoms if atom.atnum == 1]
dist_before = []
for atom in hydrogens:
    assert atom.num_bonds ==1
    d = atom.distance(atom.bonded_atoms[0])
    dist_before.append(d)

In [ ]:
hello = mdmol.calculate()


In [ ]:
hello['potential_energy']

In [ ]:
mdmol.set_integrator(mdt.integrators.LAMMPSNvt,
                   timestep=1.0*u.fs,
                   temperature=300.0*u.kelvin,
                   frame_interval=50*u.fs)

In [ ]:
traj = mdmol.run(300*u.fs)

In [ ]:
canvas = traj.draw()
canvas

In [ ]:
# numNonConstrained = 0
# for idx, atom in enumerate(hydrogens):
#     assert atom.num_bonds ==1
#     d = atom.distance(atom.bonded_atoms[0])
#     if abs(d-dist_before[idx]).value_in(u.angstrom) > 0.0001:
#         numNonConstrained += 1
# print numNonConstrained

In [ ]:
mdmol.set_integrator(mdt.integrators.LAMMPSNvt,
                   timestep=1.0*u.fs,
                   temperature=300.0*u.kelvin,
                   frame_interval=100*u.fs)

In [ ]:
traj = mdmol.run(1000*u.fs)

In [ ]:
# cavs.animate()

In [ ]:
# mdmol.set_integrator(mdt.integrators.LAMMPSNvt,
#            timestep=1.0*u.fs,
#            temperature=300.0*u.kelvin,
#            frame_interval=100*u.fs)
# thistraj = mdmol.run(1000*u.fs)
# cavs = thistraj.draw()

# cavs.animate()